In [2]:
import ast
import pandas as pd
import matplotlib.pyplot as plt
import re

In [11]:
answer_regex = re.compile('\[([-+]?[0-9]*\.?[0-9]*),\s*([-+]?[0-9]*\.?[0-9]*)\]')


def handle_valence (row):
   if row['WIDGET-TYPE'] == "emospace1" :
      return answer_regex.match(row["ANS"])[1]
    

def handle_arousal (row):
   if row['WIDGET-TYPE'] == "emospace1" :
      return answer_regex.match(row["ANS"])[2]

'-0.6613928222656247'

In [28]:

answer_data = pd.read_csv(f'BIRAFFE-procedure/BIRAFFE-procedure/SUB1107-Procedure.csv', sep=";")
answer_data = answer_data[["IADS-ID", "IAPS-ID", "WIDGET-TYPE", "ANS", "ANS-TIME"]]


answer_data["EMO-ANS"] = 0
answer_data["AROUSAL-ANS"] = 0
answer_data["VALENCE-ANS"] = 0

answer_data.loc[answer_data['WIDGET-TYPE'] == "emoscale1", 'EMO-ANS'] = answer_data["ANS"]
# answer_data.loc[answer_data['WIDGET-TYPE'] == "emospace1", 'VALENCE-ANS'] = 
# answer_data.loc[answer_data['WIDGET-TYPE'] == "emospace1", 'AROUSAL-ANS'] = answer_regex.match(answer_data[answer_data['WIDGET-TYPE'] == 'emospace1'].iloc[0]["ANS"])[2]

df['VALENCE-ANS'] = df.apply (lambda row: label_race(row), axis=1)
# answer_regex.match(answer_data[answer_data['WIDGET-TYPE'] == 'emospace1'].iloc[0]["ANS"])[1]

answer_data.head()

TypeError: expected string or bytes-like object

In [28]:
meta_data = pd.read_csv("BIRAFFE-metadata.csv", sep=";")
meta_data = meta_data.dropna(subset=['OPENNESS', 'CONSCIENTIOUSNESS', 'EXTRAVERSION',	'AGREEABLENESS', 'NEUROTICISM', 'NEO-FFI', 'PROCEDURE'])
meta_data.head()


,ID,AGE,SEX,PROCEDURE-BEGIN-TIMESTAMP,PROCEDURE-END-TIMESTAMP,BIOSIGS-BEGIN-TIMESTAMP,BIOSIGS-END-TIMESTAMP,OPENNESS,CONSCIENTIOUSNESS,EXTRAVERSION,AGREEABLENESS,NEUROTICISM,NEO-FFI,BIOSIGS,PROCEDURE,SPACE,FREUD,PHOTOS
0,1107,21.0,F,1.555651e+09,1.555655e+09,1.555651e+09,1.555655e+09,4.0,6.0,10.0,4.0,3.0,Y,Y,Y,Y,Y,Y
1,1153,22.0,F,1.555672e+09,1.555676e+09,1.555672e+09,1.555676e+09,5.0,3.0,8.0,3.0,8.0,Y,Y,Y,Y,Y,Y
2,1233,21.0,F,1.555411e+09,1.555415e+09,NaN,NaN,6.0,8.0,7.0,10.0,2.0,Y,NaN,Y,Y,Y,Y
5,1318,21.0,M,1.558009e+09,1.558014e+09,1.558009e+09,1.558014e+09,3.0,6.0,5.0,5.0,2.0,Y,Y,Y,Y,Y,Y
6,1400,22.0,M,1.554710e+09,1.554715e+09,1.554711e+09,1.554715e+09,4.0,6.0,5.0,6.0,6.0,Y,Y,Y,Y,Y,Y


In [29]:
iads_data = pd.read_csv("IADS-database.csv", sep=";")
iads_data = iads_data.rename(columns={"Number": "IADS-ID"})
iads_data.head()



,Sound,IADS-ID,ValenceMean,ValenceSD,ArousalMean,ArousalSD
0,Cat,102,4.63,2.17,4.91,1.97
1,Panting,104,4.96,1.68,5.37,1.66
2,Puppy,105,2.88,2.14,6.40,2.13
3,Growl1,106,3.37,1.64,6.39,1.62
4,Dog,107,5.47,2.22,5.85,1.81


In [30]:
iaps_data = pd.read_csv("IAPS-database.csv", sep=";")
iaps_data = iaps_data.rename(columns={"IAPS": "IAPS-ID"})
iaps_data.astype({'IAPS-ID': 'int32'})
iaps_data.head()



,Description,IAPS-ID,ValenceMean,ValenceSD,ArousalMean,ArousalSD,set
0,Snake,1019.0,3.95,1.96,5.77,1.83,12
1,Snake,1022.0,4.26,2.04,6.02,1.97,12
2,Snake,1026.0,4.09,1.91,5.61,2.23,16
3,Snake,1030.0,4.30,2.35,5.46,2.43,1
4,Snake,1033.0,3.87,1.94,6.13,2.15,18


In [31]:

def preprocess_face_data(subject_id):
    face_data = pd.read_csv(f'BIRAFFE-photo/SUB{subject_id}-Face.csv', sep=";")
    face_data = face_data.dropna(subset=['IADS-ID', 'IAPS-ID'])
    face_data = face_data.dropna(subset=['ANGER', 'CONTEMPT', 'DISGUST', 'FEAR', 'HAPPINESS', 'NEUTRAL', 'SADNESS',	'SURPRISE'])
    
    merged = face_data.merge(iaps_data, how='left', on='IAPS-ID')
    merged = merged.merge(iads_data, how='left', on='IADS-ID', suffixes=('-IAPS', '_IADS'))
    
    grouped = merged.groupby(['IADS-ID', 'IAPS-ID']).mean()
    grouped = grouped.drop(columns=["GAME-TIMESTAMP", "PICTURE-TIMESTAMP", "set"])
    return grouped


In [ ]:
def preprocess_answer_data(subject_id):
    answer_data = pd.read_csv(f'BIRAFFE-procedure/BIRAFFE-procedure/SUB{subject_id}-Procedure.csv', sep=";")
    # answer_data = answer_data.dropna(subset=['IADS-ID', 'IAPS-ID'])
    # answer_data = answer_data.dropna(subset=['ANGER', 'CONTEMPT', 'DISGUST', 'FEAR', 'HAPPINESS', 'NEUTRAL', 'SADNESS',	'SURPRISE'])
    
    # merged = answer_data.merge(iaps_data, how='left', on='IAPS-ID')
    # merged = merged.merge(iads_data, how='left', on='IADS-ID', suffixes=('-IAPS', '_IADS'))
    
    # grouped = merged.groupby(['IADS-ID', 'IAPS-ID']).mean()
    # grouped = grouped.drop(columns=["GAME-TIMESTAMP", "PICTURE-TIMESTAMP", "set"])
    # return grouped


In [34]:
for index, row in meta_data.iterrows():
    subject_id = row['ID']
    data = preprocess_face_data(subject_id)
    
    data['SUB-ID'] = subject_id
    data['OPENNESS'] = row['OPENNESS']
    data['CONSCIENTIOUSNESS'] = row['CONSCIENTIOUSNESS']
    data['EXTRAVERSION'] = row['EXTRAVERSION']
    data['AGREEABLENESS'] = row['AGREEABLENESS']
    data['NEUROTICISM'] = row['NEUROTICISM']
    
    answer_data = pd.read_csv(f'BIRAFFE-procedure/BIRAFFE-procedure/SUB{subject_id}-Procedure.csv', sep=";")

    
    
    
    data = data.dropna()
    data.to_csv(f'preprocessed/DATA-{subject_id}.csv')
    

In [35]:
import glob

path = 'preprocessed'
all_files = glob.glob(path + "/DATA-*.csv")

li = []

for filename in all_files:
    df = pd.read_csv(filename, index_col=None, header=0)
    li.append(df)

all_dfs = pd.concat(li, axis=0, ignore_index=True)
all_dfs.to_csv('preprocessed/ALL-DATA.csv')